In [1]:
RUNNING_LOCALLY = True

In [2]:
import os

# PLEASE SET YOUR OWN WORKING_DIRECTORY WHEN RUNNING LOCALLY
WORKING_DIRECTORY = "/home/yash/Desktop/Courses/CS2470/Final_Project/working_dir/"

if not RUNNING_LOCALLY:
    os.chdir("/home/yash/")
    print("Current Directory ->", os.getcwd())

    WORKING_DIRECTORY = "/home/yash/working_dir/"

    # Ensure that you are working in the right environment
    !echo $CONDA_PREFIX

LOG_FILE = WORKING_DIRECTORY + "log_file.txt"

def write_to_log(statement, include_blank_line=False):
    try:
        with open(LOG_FILE, "a") as myfile:
            if include_blank_line:
                myfile.write("\n\n" + statement)
            else:
                myfile.write("\n" + statement)
    except:
        # Running this locally may cause errors, and isn't required
        pass

In [3]:
import gym
import time
import numpy as np
from collections import deque
from gym import spaces
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import time

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2022-05-05 17:50:50.301459: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-05 17:50:50.301479: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/yash/Desktop/Courses/CS2470/Final_Project/proj_env/lib/python3.7/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


Num GPUs Available:  0


/home/yash/Desktop/Courses/CS2470/Final_Project/proj_env/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:23: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
/home/yash/Desktop/Courses/CS2470/Final_Project/proj_env/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:24: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
/home/yash/Desktop/Courses/CS2470/Final_Project/proj_env/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:25: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': pil_image.BICUBIC,
/home/yash/Desktop/Courses/CS2470/Final_Project/proj_env/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:28: DeprecationWarning: HAMMIN

In [4]:
class ConcatObs(gym.Wrapper):
    def __init__(self, env, k):
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = \
            spaces.Box(low=0, high=255, shape=((k,) + shp), dtype=env.observation_space.dtype)

    def reset(self):
        ob = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob()

    def step(self, action):
        ob, reward, done, info = self.env.step(action)
        self.frames.append(ob)
        return self._get_ob(), reward, done, info

    def _get_ob(self):
        return np.array(self.frames)

In [5]:
# A bunch of wrappers to get us started, please use these
class ObservationWrapper(gym.ObservationWrapper):
    def __init__(self, env, GRAYSCALE=False, NORMALIZE=False):
        self.GRAYSCALE = GRAYSCALE
        self.NORMALIZE = NORMALIZE
        super().__init__(env)
    
    def observation(self, obs):
        # Normalise observation by 255

        
        if self.NORMALIZE:
            obs = obs / 255.0
        # Convert to grayscale -> This isn't quite working right now, but we can update the function quite easily later
        if self.GRAYSCALE:
#             obs = obs
            obs = tf.image.rgb_to_grayscale(obs)
                    
        image = obs[:,2:-9,8:,:]
        image = tf.image.resize(image,[84,84])
        image = tf.transpose(tf.reshape(image, image.shape[:-1]),perm = [1,2,0])
        return image

class RewardWrapper(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def reward(self, reward):
        # Clip reward between 0 to 1
        return np.clip(reward, 0, 1)
    
class ActionWrapper(gym.ActionWrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def action(self, action):
        return action

In [6]:
env = gym.make("ALE/Riverraid-v5")

A.L.E: Arcade Learning Environment (version 0.7.5+db37282)
[Powered by Stella]


In [7]:
# Using the wrappers for the environment
env = ObservationWrapper(RewardWrapper(ActionWrapper(ConcatObs(env, 4))), GRAYSCALE=True, NORMALIZE=True)
obs = env.reset()

2022-05-05 17:50:58.813876: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [116]:
#visualization
#image = tf.keras.preprocessing.image.array_to_img(tf.reshape(image[:,:,3],[84,84,1]))

In [ ]:
# image1 = tf.image.resize(obs_,[84,84])
# tf.keras.preprocessing.image.array_to_img(image1)

**Episodes with n_game**

In [18]:
model_name = "random_model"

In [10]:
n_games = 1000

In [15]:
rot_list = np.zeros(n_games)
time_list = np.zeros(n_games)
num_steps = np.zeros(n_games)

for i in range(0, n_games):
    if (i+1)%10 == 0:
        print(f'Running game {i+1}/{n_games}...')
    
    obs = env.reset()
    rTot = 0
    start_time = time.time()
    done = False
    step_count = 0
    
    while not done:
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        step_count += 1
        rTot += reward

    end_time = time.time()
    inter = end_time-start_time
    rot_list[i] = rTot
    time_list[i] = inter
    num_steps[i] = step_count

env.close()

# write_to_log("average_reward: {}".format(sum(rot_list)/n_games), include_blank_line=False)
# write_to_log("average_time: {}".format(sum(time_list)/n_games), include_blank_line=False)
# write_to_log("sum(rot_list[-5:])/5: {}".format(sum(rot_list[-5:])/5), include_blank_line=False)
# write_to_log("sum(time_list[-5:])/5: {}".format(sum(time_list[-5:])/5), include_blank_line=False)

Running game 10/30...
Running game 20/30...
Running game 30/30...


In [16]:
def describe_array(arr):
    return arr.min(), arr.max(), arr.mean(), arr.std()

In [22]:
print(describe_array(num_steps))
print(describe_array(rot_list))
print(describe_array(time_list))

(503.0, 910.0, 721.8333333333334, 95.1595443919783)
(12.0, 27.0, 21.3, 3.6891733491393435)
(2.417938232421875, 4.532612562179565, 3.4751102924346924, 0.47531624347623735)


In [23]:
np.save(WORKING_DIRECTORY + model_name + "_" + str(n_games) + "_num_steps", num_steps)
np.save(WORKING_DIRECTORY + model_name + "_" + str(n_games) + "_rot_list", rot_list)
np.save(WORKING_DIRECTORY + model_name + "_" + str(n_games) + "_time_list", time_list)

In [26]:
num_steps = np.load(WORKING_DIRECTORY + model_name + "_" + str(n_games) + "_num_steps.npy")
rot_list = np.load(WORKING_DIRECTORY + model_name + "_" + str(n_games) + "_rot_list.npy")
time_list = np.load(WORKING_DIRECTORY + model_name + "_" + str(n_games) + "_time_list.npy")

In [27]:
print(describe_array(num_steps))
print(describe_array(rot_list))
print(describe_array(time_list))

(503.0, 910.0, 721.8333333333334, 95.1595443919783)
(12.0, 27.0, 21.3, 3.6891733491393435)
(2.417938232421875, 4.532612562179565, 3.4751102924346924, 0.47531624347623735)


In [12]:
print("average_reward: {}".format(rot_list.sum()/n_games))
print("average_time: {}".format(time_list.sum()/n_games))

average_reward: 0.0
average_time: 3.476018190383911


In [ ]:
print("average_reward: {}".format(sum(rot_list)/n_games))
print("average_time: {}".format(sum(time_list)/n_games))
print("sum(rot_list[-5:])/5: {}".format(sum(rot_list[-5:])/5))
print("sum(time_list[-5:])/5: {}".format(sum(time_list[-5:])/5))

1. After 100 games:<br>
   average_reward:773.95<br>
   average_time:21.668797335624696<br>

In [173]:
# q_net_copy = keras.models.load_model(model_path)

In [1]:
# q_net_copy.get_weights()